In [1]:
import pandas as pd
import numpy as np
import nltk
import toolkit
import os

from pprint import pprint
from textblob import Word
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))
toolkit.set_default_pandas_options()
pd.options.display.max_colwidth = 200

In [2]:
df = pd.read_csv('cleaned_perfume_data.csv')
df.head(100)

,Name,Brand,Description,Notes,Cleaned_Description
0,tihota eau de parfum,Indult,"Rapa Nui for sugar, Tihota is, quite simply, The One. The One that will call to you every moment you're not smelling it, The One that you've only had hints of in other vanilla perfumes, The One t...","Vanilla bean, musks",rapa nui sugar tihota quite simply one one call every moment not smell one hint vanilla perfumes one last last one perfectly capture essence pure nature fresh vanilla bean harness beyond wildest d...
1,sola parfum,Di Ser,"A tribute to the expanse of space extending from the sky to the flower and fruit laden earth below, Sola, the newest offering from DI SER and perfumer Yasuyuki Shinohara, showcases the natural fl...","Lavender, Yuzu, Lemongrass, Magnolia, Geranium, Jasmine, Frankincense, Myrrh",tribute expanse space extend sky flower fruit laden earth sola new offering di ser perfumer yasuyuki shinohara showcase natural flora hokkaido japan locally source sustainably harvest ingredient d...
2,kagiroi parfum,Di Ser,"An aromatic ode to the ancient beauty of Japan, Kagiroi represents the changes in the shade of the dawn sky over the mountains of Hokkaido, Japan. As the dawn sky transitions through shades of pu...","Green yuzu, green shikuwasa, sansho seed, coriander, ylang-ylang, shiso, rosewood, vetiver, hinoki, cypriol, patchouli, agarwood",aromatic ode ancient beauty japan kagiroi represent change shade dawn sky mountain hokkaido japan dawn sky transition shade purple yellow orange red scent showcase many ingredient use first time p...
3,velvet fantasy eau de parfum,Montale,"Velvet Fantasy is a solar fragrance where citrus and velvety fruits mingle with the elegance of rose and the gourmand femininity of violet. Thereafter, incense unveils amber and white musk to cre...","tangerine, pink pepper, black coffee, leather, violet, jasmine, lily of the valley, heliotrope powder, vanilla, amber, sandalwood, toffee, musk, oakmoss",velvet fantasy solar fragrance citrus velvety fruit mingle elegance rise gourmand femininity violet thereafter incense unveil amber white musk create bewitching wild dance think sweet fruity fragr...
4,a blvd. called sunset eau de parfum,A Lab on Fire,"There's no way A Lab On Fire could relocate to Los Angeles and not pay tribute to L.A.'s most iconic Boulevard, home to the legendary neon-drenched rock stages of the Strip, the crashing waves of...","Bergamot, almond, violet, jasmine, leather, sandalwood, vanilla, tonka",no way lab fire could relocate los angeles not pay tribute las iconic boulevard home legendary neondrenched rock stage strip crash wave palisade everything result blvd call sunset bright fresh rig...
5,freckled and beautiful eau de parfum,A Lab on Fire,"There's no beauty quite like being young in the City of Angels- the wind in your hair, the sand between your toes, the endless possibilities for freedom and adventure in a sun-dappled, ocean-kiss...","Orange flower, neroli, honeysuckle, warm milk, pastry, salicylates, sandalwood, vanilla bean, heliotrope",no beauty quite like young city angel wind hair sand toe endless possibility freedom adventure sundapple oceankisse paradise day never end anything possible freckle beautiful perfect tribute beaut...
6,exit the king eau de parfum,Etat Libre d'Orange,"In these tumultuous times, a fragrance that celebrates destruction and dethronement might seem... risky. But Etat Libre was never about playing it safe, and Exit The King, like all their best rel...","Timur JE, Soap Foam Accord (Aldehydes & Musk), Pink Pepper, Jasmine e-pure, Rose Superessence, Lily-of-the-valley Accord, Patchouli, Moss Absolute, Sandalwood Accord & Orcanox",tumultuous time fragrance celebrate destruction dethronement may seem risky etat libre never play safe exit king like good release risky good ways haveneversmelledanythingquitelikethis fragrance t...
7,eshu extrait,PRIN,"Eshu, the mischievous god of Yoruba legend, who translates between the gods and all the peoples of earth. Es

In [3]:
cleaned_description = df['Cleaned_Description']

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

#build BOW Term Frequency features
bow_tf = CountVectorizer(binary=False, min_df=2, max_df=1.0)
bow_tf_matrix = bow_tf.fit_transform(cleaned_description)

#dense representation
bow_tf_matrix = bow_tf_matrix.toarray()

In [5]:
#build BONG Term Frequency features [Uni & Bi-grams]
bong_tf = CountVectorizer(binary=False, ngram_range=(1,2))
bong_tf_matrix = bong_tf.fit_transform(cleaned_description)

#dense representation 
bong_tf_matrix = bong_tf_matrix.toarray()

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

#build BOW Term Frequency-Inverse Document Frequency features
bow_tfidf = TfidfVectorizer(use_idf=True, min_df=2, max_df=1.0)
bow_tfidf_matrix = bow_tfidf.fit_transform(cleaned_description)

#dense representation
bow_tfidf_matrix = bow_tfidf_matrix.toarray()

In [7]:
# build BONG Term Frequency-Inverse Document Frequency features [Uni & Bi-grams] 
bong_tfidf = TfidfVectorizer(ngram_range=(1,2), use_idf=True, min_df=2, max_df=1.0)
bong_tfidf_matrix = bong_tfidf.fit_transform(cleaned_description)

#dense representation
bong_tfidf_matrix = bong_tfidf_matrix.toarray()

In [8]:
from sklearn.metrics.pairwise import cosine_similarity

#document similarity 1, bow model-based vectors with term frequency values
bow_tf_similarity_matrix = cosine_similarity(bow_tf_matrix)
bow_tf_similarity_df = pd.DataFrame(bow_tf_similarity_matrix)

In [9]:
#document similarity 2, bow model-based vectors with TFIDF values 
bow_tfidf_similarity_matrix = cosine_similarity(bow_tfidf_matrix)
bow_tfidf_similarity_df = pd.DataFrame(bow_tfidf_similarity_matrix)

In [10]:
#document similarity 3, bong model-based vectors with term frequency values
bong_tf_similarity_matrix = cosine_similarity(bong_tf_matrix)
bong_tf_similarity_df = pd.DataFrame(bong_tf_similarity_matrix)

In [11]:
#document similarity 4, bong model-based vectors with TFIDF values 
bong_tfidf_similarity_matrix = cosine_similarity(bong_tfidf_matrix)
bong_tfidf_similarity_df = pd.DataFrame(bong_tfidf_similarity_matrix)

In [12]:
#perfume recommender
#The perfume name, perfume list and document similarity matrix dataframe will be given as inputs to the function
def perfume_recommender(df, perfume_name, perfume_sims):
    perfumes = df['Name'].array
    notes = df['Notes'].array
    # find perfume row index
    perfume_idx = np.where(perfumes == perfume_name)[0][0]
    # get perfume similarities
    perfume_similarities = perfume_sims.iloc[perfume_idx].values
    # get top 5 similar perfume row index
    similar_perfume_idxs = np.argsort(-perfume_similarities)[1:6]
    # get top 5 perfumes and notes
    similar_dict = {df.Name.iloc[idx]: df.Notes.iloc[idx] for idx in similar_perfume_idxs}
    # return the top 5 perfumes and notes
    return similar_dict

In [13]:
#evaluation of recommender systems with various feature engineering models
perfume_name = ['spicematic eau de parfum', 'antico caruso eau de parfum']
perfume_sims = {'bow_tf': bow_tf_similarity_df,
                 'bow_tfidf': bow_tfidf_similarity_df,
                 'bong_tf':bong_tf_similarity_df,
                 'bong_tfidf':bong_tfidf_similarity_df}

for perfume in perfume_name:
    print('Perfume:', perfume)
    print('\n')
    print(df.loc[df['Name'] == perfume, ['Name', 'Notes']])
    print('\n\n')
    
    for k,v in perfume_sims.items():
        print('Top 5 recommended Perfumes using', k, ':\n')
        output = perfume_recommender(df, perfume, v)
        outputdf = pd.DataFrame()
        outputdf['Perfume'] = output.keys()
        outputdf['Notes'] = output.values()
        print(outputdf)
        print('\n')
    print('\n')

Perfume: spicematic eau de parfum


                         Name                                             Notes
398  spicematic eau de parfum   Saffron, ginger, mint, incense, red cedar, pine



Top 5 recommended Perfumes using bow_tf :

                        Perfume                                                                                              Notes
0     indian wood eau de parfum                  Sandalwood, vetiver, moss, spicy mint, nutmeg, cardamom, coconut milk, lemon zest
1                   nordic noir   Cardamom, rosemary, spearmint, mint, orris, ginger, heliotrope, cedar, sandalwood, vanilla, musk
2     just before eau de parfum      Bergamot, lemon, saffron, palo santo, ylang-ylang, myrrh, vanilla, benzoin, amber, sandalwood
3  menthe fraiche eau de parfum                         mint leaves, Sicilian bergamot, mate, green tea, lotus leaves, white cedar
4    peau d'ambre eau de parfum                                        Frankincense, fir balsam, opopon

In [14]:
#final recommendations for a list of perfumes using the BONG-TF Model
perfume_name = ['yuzu eau de parfum', 'dolce acqua eau de parfum']
perfume_sims = {'bong_tf': bong_tf_similarity_df}

for perfume in perfume_name:
    print('Perfume:', perfume)
    print('\n')
    print(df.loc[df['Name'] == perfume, ['Name', 'Notes']])
    print('\n\n')
    
    for k,v in perfume_sims.items():
        print('Top 5 recommended Perfumes: ')
        output = perfume_recommender(df, perfume, v)
        outputdf = pd.DataFrame()
        outputdf['Perfume'] = output.keys()
        outputdf['Notes'] = output.values()
        print(outputdf)
    print('\n')

Perfume: yuzu eau de parfum


                   Name                                                                    Notes
664  yuzu eau de parfum   Lemon, Bergamot, Orange, Thyme, Grapefruit, Lime, Yuzu, Rose, Mandarin



Top 5 recommended Perfumes: 
                               Perfume                                                                                                                                            Notes
0           note de yuzu eau de parfum                                                                                            Yuzu, lemon, mandarine, sea salt, vetiver, white musk
1                   shiragoromo parfum                                                                                                             Agarwood, yuzu, jasmine sambac, rose
2         italian citrus eau de parfum                     Cold pressed lemon, chinotto, blood orange, green mandarin, violet leaf, incense, copaiba balsam, musk ambrette, and oakmoss
3       